# Домашее задание2. Линейная регрессия для BostonHousePrices

Владимир Никифоров

In [1]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
boston = load_boston()

In [3]:
X = pd.DataFrame(boston['data'],columns=boston['feature_names'])
y = boston['target']
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, X_test.shape)
scl = StandardScaler()
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)
X_train

(404, 13) (102, 13)


array([[-0.42490634,  3.52765396, -1.2687873 , ..., -1.75380084,
         0.37495612, -1.36715761],
       [-0.25351295, -0.48489298,  1.53629686, ...,  1.25651421,
         0.44399331,  3.04004516],
       [-0.41718882,  0.7822271 , -0.94023762, ..., -0.87386259,
         0.30135269, -0.73955404],
       ...,
       [-0.42222338, -0.48489298,  0.08191694, ...,  1.16388914,
         0.44399331, -0.50053353],
       [-0.42117065,  0.90893911, -1.34179834, ..., -0.04023688,
         0.40898547, -0.8010564 ],
       [-0.40508883,  0.44432841, -0.80443709, ...,  0.28395089,
         0.17860783, -0.02249251]])

In [12]:
batch_size = 32
lr = 0.005
features = torch.FloatTensor(X_train)
labels = torch.FloatTensor(y_train)
dataset = TensorDataset(features, labels)
data_iter = DataLoader(dataset, batch_size, shuffle=True)

In [13]:
model = torch.nn.Sequential(torch.nn.Linear(13, 42),
                            torch.nn.ReLU(),
                            torch.nn.Linear(42, 42),
                            torch.nn.ReLU(),
                            torch.nn.Linear(42, 1)
                           )

In [14]:
trainer = torch.optim.Adam(model.parameters(), lr=lr)

In [15]:
loss = torch.nn.MSELoss(reduction='mean')

In [16]:
num_epochs = 2000
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        trainer.zero_grad()
        l = loss(model.forward(X).reshape(-1), y)
        l.backward()
        trainer.step()
    l = loss(model.forward(features).reshape(-1), labels)
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 462.350769
epoch 2, loss: 130.433395
epoch 3, loss: 58.696667
epoch 4, loss: 29.579668
epoch 5, loss: 22.990805
epoch 6, loss: 18.946651
epoch 7, loss: 17.050625
epoch 8, loss: 15.681053
epoch 9, loss: 14.602250
epoch 10, loss: 13.798654
epoch 11, loss: 13.086932
epoch 12, loss: 12.579563
epoch 13, loss: 12.019958
epoch 14, loss: 11.662828
epoch 15, loss: 11.348551
epoch 16, loss: 10.959620
epoch 17, loss: 10.832983
epoch 18, loss: 10.561760
epoch 19, loss: 10.268744
epoch 20, loss: 10.274724
epoch 21, loss: 9.817596
epoch 22, loss: 9.613408
epoch 23, loss: 10.091314
epoch 24, loss: 9.638775
epoch 25, loss: 9.306218
epoch 26, loss: 9.249727
epoch 27, loss: 9.845023
epoch 28, loss: 9.231816
epoch 29, loss: 8.843622
epoch 30, loss: 8.800476
epoch 31, loss: 8.625225
epoch 32, loss: 8.643315
epoch 33, loss: 8.514500
epoch 34, loss: 8.340572
epoch 35, loss: 8.284242
epoch 36, loss: 8.539547
epoch 37, loss: 8.124536
epoch 38, loss: 8.129226
epoch 39, loss: 8.222919
epoch 40, l

In [17]:
model.eval()

seq = torch.FloatTensor(X_test)
with torch.no_grad():
    y_pred = model(seq)

In [18]:
print(y_test)
print(f'RMSE on test: {mean_squared_error(y_test, y_pred.numpy())**0.5}')

[22.6 11.9 50.  35.1 18.8 18.6 24.3 24.7 13.8 22.  12.5 23.  15.2 28.2
 13.5 17.1 14.5 29.  15.7 12.1 37.2 33.8 17.5 19.6 19.7  8.3 22.4  8.8
 23.7 18.5 15.6 29.  21.4 27.9 14.3 23.1 10.8 22.6 21.7 27.1 22.6 19.6
 19.8 27.1 11.5 12.3 18.8 25.  21.4  9.5 22.  32.  23.8 18.3 43.1 28.5
 20.5 36.2 18.4 26.6 21.1 16.7 18.4 46.7 17.1 22.2 48.8 18.5 30.1 23.1
 17.3 46.  16.6 26.4 13.4 34.9 19.4 19.5 21.7 21.  20.9 39.8 16.1 16.6
 50.  19.5 48.5 28.7 22.9 22.9 22.9 26.7 23.3 32.  13.2 12.8 11.7 29.1
 18.7 18.3 31.2 23.6]
RMSE on test: 3.937717939407795
